In [9]:
%matplotlib inline
import pandas
import matplotlib
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

# Average window_stride elements together to form a single row
window_stride = 12

sample_hours = window_stride / 12.0
print("Sample Hours: %f" % sample_hours)

# Number of future samples to mean for prediction
prediction_window = int(24 / sample_hours)
print("Prediction Window: %d" % prediction_window)

# Length of the windowed sequence
sequence_length = int(7*24 / sample_hours)
print("Sequence Length: %d" % sequence_length)

# Input Features
input_columns = ['epoch', 'hour', 'temp', 'windspd', 'winddir', 'no', 'no2', 'nox', 'o3']
output_columns = ['no', 'no2', 'nox', 'o3']

# Take the FFT of each sqeuence and use as features
fft_features = False

# Fit the sequence to y = mx+b and add the coeff / intercept
regression_features = True

# Add variance for each feature in the sequence
std_features = True

input_map = {value: idx for idx, value in enumerate(input_columns)}
output_map = {value: idx for idx, value in enumerate(output_columns)}

# Number of features we take from the data
num_inputs = len(input_columns)

# Number of things we are doing regression to predict
num_outputs = len(output_columns)

# All the timesteps features unrolled length
unrolled_input_length = num_inputs * sequence_length

train_validation_split = 0.33

Sample Hours: 1.000000
Prediction Window: 24
Sequence Length: 168


In [11]:
# ----- Feature Extraction ------

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

# Read the data
df = pandas.read_csv('d00.csv')

# Drop useless columns
df = df.drop(['AQS_Code', 'Latitude', 'Longitude', 'day'], axis=1)

# Move from pandas to numpy for windowing
nd = df[input_columns].values

# Windowed dataset
nd_window = np.zeros((int(nd.shape[0] / window_stride), num_inputs))

row = 0
while row < nd.shape[0] - window_stride:
    for i in range(0, num_inputs):
        nd_window[int(row/window_stride)][i] = np.mean(nd[row:row+window_stride,i])
    row += window_stride
    
# Scale the values between 0 and 1
scaler = MinMaxScaler()
scaler.fit(nd_window)
nd_window = scaler.transform(nd_window)

# Create sequences for training
data = []
labels = []

rows = deque(maxlen=sequence_length)

for idx, r in enumerate(nd_window):

    rows.append([f for f in r])
    
    # We need the entire sequence filled to make a prediction about the future mean
    if len(rows) < sequence_length:
        continue
    
    # Since we are predicting the mean, make sure we do not go out of bounds in the future
    if idx+1 + prediction_window > nd_window.shape[0]:
        break
                
    data.append(list(rows).copy())
        
    # We are predicting the future mean values
    u_24_no = np.mean( nd_window[idx+1 : idx+1 + prediction_window, input_map['no']] )
    u_24_no2 = np.mean( nd_window[idx+1 : idx+1 + prediction_window, input_map['no2']] )
    u_24_nox = np.mean( nd_window[idx+1 : idx+1 + prediction_window, input_map['nox']] )
    u_24_o3 = np.mean( nd_window[idx+1 : idx+1 + prediction_window, input_map['o3']] )
    
    labels.append([u_24_no, u_24_no2, u_24_nox, u_24_o3])

# Add some features
for idx, r in enumerate(data):
    
    features_to_add = []
        
    if fft_features:
                
        bins = np.abs(np.fft.fft(r))
        bins = bins[:int(len(bins)/2)]

        fft_features = len(bins)

        features_to_add.extend(np.array(bins[:, 2:8]).ravel().tolist())
        
        new_r.extend(features_to_add)
        
    if regression_features:
        for f in range(2, num_inputs):

            X = [[i] for i in range(0, sequence_length)]
            Y = [[i] for i in np.array(r)[:, f].tolist()]
            
            lr = LinearRegression().fit(X, Y)
            
            features_to_add.extend([lr.coef_[0][0], lr.intercept_[0]])
            
    if std_features:
        for f in range(2, num_inputs):
            features_to_add.append(np.std(np.array(r)[:, f]))

    new_r = np.array(r).ravel().tolist()
    new_r.extend(features_to_add)
    
    data[idx] = new_r

data = np.array(data)
labels = np.array(labels)

print("--data--")
print(data.shape)
print("--labels--")
print(labels.shape)

# Train validation split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.33, random_state=42)

--data--
(8142, 1362)
--labels--
(8142, 4)


In [ ]:
# Training
import pickle
from sklearn.ensemble import RandomForestRegressor

best_r2 = None

for epoch in range(0, 100):
    regr = RandomForestRegressor(random_state=epoch, n_estimators=100, n_jobs=-1, verbose=0)
    regr.fit(X_train, y_train)
    r2 = regr.score(X_test, y_test)
    
    save = False
    
    if best_r2 is None:
        print("epoch(%d) - R^2: %f" % (epoch+1, r2))
        best_r2 = r2
        save = True
    elif r2 > best_r2:
        print("epoch(%d) - R^2 improved: %f (best: %f)" % (epoch+1, r2, best_r2))
        best_r2 = r2
        save = True
    else:
        print("epoch(%d) - R^2 did not improve: %f (best: %f)" % (epoch+1, r2, best_r2))
    
    if save:
        open('random-forest.best.pickle', 'wb').write(pickle.dumps(regr))

epoch(1) - R^2: 0.885263


In [ ]:
# FFT Visualization

plt.rcParams["figure.figsize"] = (20, 10)
plt.rcParams['font.size'] = 15

bins = np.abs(np.fft.fft(data[400][:unrolled_input_length].reshape((sequence_length, num_inputs))))
freqs = np.fft.fftfreq(len(bins), d=300*12)[:int(len(bins)/2)]
bins = bins[:int(len(bins)/2)]

for key in output_map:
    plt.plot(freqs, bins[:,output_map[key]])
    
plt.legend([output_columns[idx] for idx in range(0, num_outputs)])
plt.title("Windowed Data FFT")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude (Scaled)")
plt.show()

In [11]:
# Visual Simulation

plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['font.size'] = 16

for seq in range(0, data.shape[0] - sequence_length):
    
    lookup = {'no': (0, 0), 'no2':(0, 1), 'nox':(1, 0), 'o3':(1, 1)}

    pred = regr.predict(data[seq].reshape((1, unrolled_input_length + fft_features*4)))[0]
    fig, ax = plt.subplots(2, 2)

    for idx, f in enumerate([(5, 'no'), (6, 'no2'), (7, 'nox'), (8, 'o3')]):
    
        feature_index, feature_name = f
        
        X = []
        Y_actual = []

        for i in range(0, sequence_length + int(24*(1/sample_hours))):
            X.append(seq+i)
            Y_actual.append(data[seq+i][feature_index])

        Y_actual = np.array(Y_actual)
        
        predicted_mean = pred[feature_index - 5]
        actual_mean = np.mean(Y_actual[sequence_length:])
        rolling_mean = np.mean(Y_actual[:sequence_length])
        rolling_std = np.std(Y_actual[:sequence_length])
                
        Y_pred = Y_actual.copy()
        Y_pred[sequence_length:] = predicted_mean
        Y_pred[:sequence_length] = np.nan

        Y_actual_mean = Y_actual.copy()
        Y_actual_mean[sequence_length:] = actual_mean
        Y_actual_mean[:sequence_length] = np.nan
        
        Y_rolling_mean = Y_actual.copy()
        Y_rolling_mean[:sequence_length] = rolling_mean
        Y_rolling_mean[sequence_length:] = np.nan
        
        Y_rolling_std_upper = Y_actual.copy()
        Y_rolling_std_upper[:sequence_length] = rolling_mean + rolling_std
        Y_rolling_std_upper[sequence_length:] = np.nan
        
        Y_rolling_std_lower = Y_actual.copy()
        Y_rolling_std_lower[:sequence_length] = rolling_mean - rolling_std
        Y_rolling_std_lower[sequence_length:] = np.nan   
        
        subplot = ax[lookup[feature_name][0]][lookup[feature_name][1]]

        subplot.plot(X, Y_actual, color='black', linewidth=4.0)
        subplot.plot(X, Y_actual_mean, color='green', linewidth=4.0)
        subplot.plot(X, Y_pred, color='purple', linewidth=4.0)
        subplot.plot(X, Y_rolling_mean, color='green', linewidth=4.0)
        subplot.plot(X, Y_rolling_std_upper, color='orange', linewidth=4.0)
        subplot.plot(X, Y_rolling_std_lower, color='orange', linewidth=4.0)
        
        subplot.grid()
        
        subplot.set_title("%s 24 hour mean prediction" % (feature_name,))
        
        subplot.set_xlabel("Hours")
        subplot.set_ylabel("Scaled Concentration")
    
    fig.legend(['Actual Continuous', 'Actual Mean', 'Predicted Mean', 'Rolling Mean', 'Standard Deviation'])
    fig.tight_layout()

    plt.savefig('charts/%.05d.png' % seq)
    # plt.show()
    plt.close()

    if seq % 100 == 0:
        print("Rendered %d" % seq)

NotFittedError: Estimator not fitted, call `fit` before exploiting the model.